[exercises](rir.ipynb)

Directly listening to a room impulse response (RIR) doesn't reveal much information (except probably for room acoustics experts).
It's normally more helpful to use a bunch of dry recordings with different characteristics (speech, music, tonal, percussive, ...), convolve them with the given RIR and listen to the results.

In [12]:
import numpy as np
from scipy import signal
import soundfile as sf
import tools

In [13]:
speech, fs = sf.read("data/xmas.wav")
rir, fs_rir = sf.read("data/rir_clap.wav")
assert fs == fs_rir
speech_clap = signal.fftconvolve(speech, rir)
# normalize to the same maximum value as the original speech signal:
speech_clap = tools.normalize(speech_clap, np.max(np.abs(speech)))
sf.write("data/xmas_clap.wav", speech_clap, fs)

<audio src="data/xmas.wav" controls></audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/rir_clap.wav" controls></audio>
[data/rir_clap.wav](data/rir_clap.wav)

<audio src="data/xmas_clap.wav" controls></audio>
[data/xmas_clap.wav](data/xmas_clap.wav)

It doesn't sound exactly like the measured room, because the frequency response of the clapping device is not flat and its characteristics are part of the measured RIR and therefore also audible in the convolved signal.

In [14]:
%matplotlib
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg


In [15]:
t = np.arange(len(rir)) / fs
plt.plot(t, rir)
plt.xlabel("time / seconds");

In [7]:
plt.figure()
plt.plot(t, tools.db(rir))
plt.xlabel("time / seconds");

In [8]:
def plot_impulse_response(ir, fs=44100, db=True):
    L = len(ir)
    time = np.arange(L) / fs * 1000
    
    plt.figure()
    if db:
        plt.plot(time, tools.db(ir))
        plt.ylabel('Amplitude / dB')
    else:
        plt.plot(time, ir)
        plt.ylabel('Amplitude')
    plt.xlabel('Time / ms')
    plt.grid()
    return

In [9]:
def energy_decay_curve(ir):
    """Normalized energy decay curve (EDC) of the impulse response.
    """
    L = np.zeros_like(ir)
    L[-1] = ir[-1]**2
    for n in range(len(ir)-2, -1, -1):
        L[n] = L[n+1] + ir[n]**2
    return L / L[0]

Notice that the energy decay curve can be interpreted as a convolution of $h^2(t)$ and a rectangular window.
Therefore, it can be computed more efficiently using the fast convolution.

In [10]:
def fast_edc(ir):
    L = len(ir)
    window = np.ones_like(ir)
    L = signal.fftconvolve(ir**2, window)[L-1:]
    return L / L[-1]

Of course, this will be efficient only if the length of the impulse response is sufficiently long.

In [11]:
import io
import soundfile as sf
import tools
import zipfile

In [8]:
url = "http://legacy.spa.aalto.fi/projects/poririrs/wavs/omni.zip"
filename = "s1_r1_o.wav"
zf = zipfile.ZipFile(tools.HttpFile(url))
pori, fs = sf.read(io.BytesIO(zf.read(filename)))

# you can also just download and unzip the file manually:
#pori, fs = sf.read(filename)

assert pori.shape[1] == 2  # stereo IR
pori = pori.sum(axis=1)
fs

48000

The sampling frequencies of the input signal and the impulse response have to match!

It's very easy to convert between sampling frequencies with [SoX](http://sox.sourceforge.net/), e.g. like this:

    sox xmas.wav -r 48000 xmas48k.wav

In [9]:
speech48k, fs48k = sf.read("data/xmas48k.wav")
assert fs48k == 48000

In [10]:
speech_pori = signal.fftconvolve(speech48k, pori)
speech_pori = tools.normalize(speech_pori, np.max(np.abs(speech)))

In [11]:
sf.write("data/xmas_pori.wav", speech_pori, fs)

<audio src="data/xmas.wav" controls></audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_pori.wav" controls></audio>
[data/xmas_pori.wav](data/xmas_pori.wav)

In [12]:
t = np.arange(len(pori)) / fs
plt.figure()
plt.plot(t, pori)
plt.xlabel("time / seconds");

In [13]:
# TODO: use custom plotting function
plt.figure()
plt.plot(t, tools.db(pori))
plt.xlabel("time / seconds");

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>